In [34]:
import pandas as pd
from docx import Document 

In [35]:
data = pd.read_excel("../Base_filtrada.xlsx")

### Separação por natureza dos dados

In [36]:
colunas_categoricas = ['V0001', 'A02201', 'C004', 'C006', 'C00701', 'C00702', 
                       'C00703', 'C009', 'L01701', 'L018', 'M01601', 'N010', 'P00102','P034',
                       'P050', 'P051', 'P052', 'P05401', 'P05404', 'P05407', 'P05410',
                       'P05413', 'P05416', 'P05419', 'P058', 'Q09201', 'Q11006', 
                       'Q11604', 'W001', 'Q074']

In [37]:
colunas_numericas = ['V0022', 'A02305', 'A02306', 'C008', 'P00103', 'P00104', 'P035', 'P03701', 'P03702', 
                     'P053', 'P05402', 'P05403', 'P05405', 'P05406', 'P05408', 'P05409',
                     'P05411', 'P05412','P05414', 'P05415', 'P05417', 'P05418', 
                     'P05421', 'P05422', 'P05801', 'P05802', 'P05901', 'P05902', 
                     'P05903', 'P05904', 'W00101', 'W00102', 'W00103']  

garantir que as colunas numericas sao numericas

In [38]:
for coluna in colunas_numericas:
    data[coluna] = pd.to_numeric(data[coluna], errors='coerce')

## Tabela numerica

In [39]:
estatisticas_numericas = {
    'Coluna': colunas_numericas,
    'Mínimo': [round(data[col].min(), 2) for col in colunas_numericas],
    'Máximo': [round(data[col].max(), 2) for col in colunas_numericas],
    'Média': [round(data[col].mean(), 2) for col in colunas_numericas],
    'Desvio Padrão': [round(data[col].std(), 2) for col in colunas_numericas],
    'Dados Faltantes': [data[col].isnull().sum() for col in colunas_numericas],
    '% Faltantes': [round((data[col].isnull().sum() / len(data)) * 100, 2) for col in colunas_numericas]
}
df_estatisticas_numericas = pd.DataFrame(estatisticas_numericas)

In [40]:
df_estatisticas_numericas = pd.DataFrame(estatisticas_numericas)

## Tabela Categorica

In [41]:
estatisticas_categoricas_detalhadas = {
    'Coluna': [],
    'Categoria': [],
    'Contagem': [],
    '% da Coluna': []
}

for coluna in colunas_categoricas:
    # Extrair categorias e contagens (incluindo NaN)
    contagem = data[coluna].value_counts(dropna=False).reset_index()
    contagem.columns = ['Categoria', 'Contagem']
    
    # Converter categorias para numérico (se possível) e ordenar
    try:
        contagem['Categoria_Num'] = pd.to_numeric(contagem['Categoria'], errors='coerce')
        contagem = contagem.sort_values(by='Categoria_Num').drop('Categoria_Num', axis=1)
    except:
        contagem = contagem.sort_values(by='Categoria')  # Caso não sejam números
    
    # Adicionar dados faltantes no final
    faltantes = data[coluna].isnull().sum()
    total_linhas = len(data)
    
    for _, row in contagem.iterrows():
        categoria = str(row['Categoria'])
        estatisticas_categoricas_detalhadas['Coluna'].append(coluna)
        estatisticas_categoricas_detalhadas['Categoria'].append(categoria)
        estatisticas_categoricas_detalhadas['Contagem'].append(row['Contagem'])
        estatisticas_categoricas_detalhadas['% da Coluna'].append(round((row['Contagem'] / total_linhas) * 100, 2))
    
    # # Adicionar linha de faltantes (se houver)
    # if faltantes > 0:
    #     estatisticas_categoricas_detalhadas['Coluna'].append(coluna)
    #     estatisticas_categoricas_detalhadas['Categoria'].append('NaN')
    #     estatisticas_categoricas_detalhadas['Contagem'].append(faltantes)
    #     estatisticas_categoricas_detalhadas['% da Coluna'].append(round((faltantes / total_linhas) * 100, 2))

# Criar DataFrame
df_categoricas_ordenado = pd.DataFrame(estatisticas_categoricas_detalhadas)


## Analise

In [42]:
print("Análise Descritiva do Conjunto de Dados Numéricos")
print(df_estatisticas_numericas)

print("Análise Descritiva do Conjunto de Dados Categóricos")


Análise Descritiva do Conjunto de Dados Numéricos
    Coluna  Mínimo  Máximo  Média  Desvio Padrão  Dados Faltantes  % Faltantes
0    V0022     1.0    23.0   2.95           1.51                0         0.00
1   A02305     0.0    40.0   0.77           1.58            36941        40.66
2   A02306     0.0    83.0   1.51           1.55            36941        40.66
3     C008    15.0   107.0  46.39          17.60                0         0.00
4   P00103    23.0   190.0  72.09          15.28             7773         8.56
5   P00104    25.0   176.0  71.88          15.16              892         0.98
6     P035     0.0     7.0   3.37           1.93            54448        59.93
7   P03701     0.0    15.0   1.01           0.84            55650        61.26
8   P03702     0.0    59.0  13.81          16.87            55650        61.26
9     P053     2.0    90.0  16.80           6.32            58783        64.71
10  P05402     1.0    80.0  12.64           9.39            83786        92.23
11

## Salvando em arquivo

In [43]:
doc = Document()
doc.add_heading('Análise Descritiva do Conjunto de Dados', 0)

In [44]:
doc.add_heading('Estatísticas Numéricas', level=1)
tabela_numericas = doc.add_table(rows=1, cols=len(df_estatisticas_numericas.columns))

In [45]:
hdr_cells = tabela_numericas.rows[0].cells

for i, col in enumerate(df_estatisticas_numericas.columns):
    hdr_cells[i].text = str(col)


In [46]:
for index, row in df_estatisticas_numericas.iterrows():
    row_cells = tabela_numericas.add_row().cells
    for i, value in enumerate(row):
        row_cells[i].text = str(value)

In [47]:
doc.add_heading('Análise Categórica', level=1)

for coluna in colunas_categoricas:
    doc.add_heading(f'Coluna: {coluna}', level=2)
    dados_coluna = df_categoricas_ordenado[df_categoricas_ordenado['Coluna'] == coluna]
    
    tabela = doc.add_table(rows=1, cols=3)
    hdr_cells = tabela.rows[0].cells
    hdr_cells[0].text = 'Categoria'
    hdr_cells[1].text = 'Contagem'
    hdr_cells[2].text = '% da Coluna'
    
    for _, row in dados_coluna.iterrows():
        row_cells = tabela.add_row().cells
        row_cells[0].text = str(row['Categoria'])
        row_cells[1].text = str(row['Contagem'])
        row_cells[2].text = f"{row['% da Coluna']}%"

In [48]:
doc.save('../Relatorios/tabela_estatisticas.docx')